# Measuring Water Level from river Elbe


##### To measure water level from river elbe, a webside(https://pegelonline.wsv.de/webservice/guideRestapi) was used where with the help of RestAPI different information were gathered in Json format. The task included converting nested list to dictionary data format and then transfering them to a dataframe. 

##### From the data frame necessary information were gathered related to rivers and their current water level in real-time.

##### Most of the sections in the beginning of the code were implemented to process the data.

In [ ]:

import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import schedule
import time
from collections import ChainMap


# A while loop to run the whole process again and again
while(True):
    try:
        p=requests.get('https://pegelonline.wsv.de/webservices/rest-api/v2/stations.json?includeTimeseries=true&includeCurrentMeasurement=true')
        soup=BeautifulSoup(p.content,"html.parser")
        script= soup.text
        data = json.loads(script)
        df=pd.DataFrame(data)
        df=pd.json_normalize(data, max_level=1)
        ##############################################
        
        #the column'timeseries' has multiple nested information. a for loop was used to convert 
        #the nested list information into dictionary and then the informations were saved in a data frame.
        
        multipleDct_to_single=[]

        for i in range(len(df['timeseries'])):
            qq=dict(ChainMap(*df['timeseries'][i][::-1 ]))
            multipleDct_to_single.append(qq)
        pd.DataFrame(multipleDct_to_single)

        #####################################
        
        # processing the nested dictionary to data frame
        
        df_time_s= pd.json_normalize(multipleDct_to_single, max_level=1)

        # Dropping less important columns
        
        df_timeseries = df_time_s.drop(['shortname','longname'],axis=1)

        #merging two dataframe into one to get all the information together
        df_all = pd.concat([df,df_timeseries],axis=1)
        
        #ELBE was filtered from all the rivers so that only datas related to ELBE are present.
        df_EL = df_all[(df_all['water.longname'] == 'ELBE')]


        ########################################################
        # Dropping less important columns
        df_ELBE = df_EL.drop(['shortname','timeseries','water.shortname','comment.shortDescription','currentMeasurement.stateNswHsw','comment.longDescription','currentMeasurement.stateNswHsw','comment.shortDescription','comment.longDescription','gaugeZero.unit'], axis=1)
        # saving the .csv file with all the related information of Elbe from all the available datapoint
        df_ELBE.to_csv('waterlevel_elbe.csv')

        ########################################################
        
        # three locations were selected from where data of ELBE river can be selected.
        
        df_ELBE_3measure_point = df_ELBE[df_ELBE['longname'].isin(['SCHÖNA', 'PIRNA', 'MEISSEN'])]

        # Saving the  collected data in a .csv file
        df_ELBE_3measure_point.to_csv('waterlevel_elbe_3points.csv')
        
        # timer is set to update the csv file every 5 minutes. here time is in seconds
        time.sleep(300)
    except():
        print('Error!')
